### This notebook consist of code for creating the html files for the website each time data is updated.

##### Set-up

In [1]:
# Change the working directory to the desired location
# temporary, only for local
import os
#os.chdir("/home/bumblebealu/groupwebsite_generator/")
os.chdir("/Users/harshul/website clone/harshul/test/groupwebsite_generator")
os.getcwd()

'/Users/harshul/website clone/harshul/test/groupwebsite_generator'

In [2]:
#Importing classes from the Jinja2 library to load and render templates.
import json
import os
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path


In [3]:
#Function for creating proper html file names
def page_link(a):
    if ' ' in a:
        return a.replace(' ', '_')
    else:
        return a

In [4]:
from pathlib import Path
import json

MEMBERS_DIR_PATH = Path('../group-data/members/')
member_records = []

file_names = [
    'experiences.json',
    'education.json',
    'awards.json',
    'roles.json',
    'projects.json',
    'website_media.json',
    'posters.json',
    'publications.json',
    'outreach.json'
]

for member_dir in MEMBERS_DIR_PATH.glob('*'):
    member_record = json.load(open(member_dir / 'info.json'))
    member_json_dir = member_dir / 'jsons'
    
    current_position = {}
    for file_name in file_names:
        file_path = member_json_dir / file_name
        if file_path.exists():
            current_position = json.load(open(file_path))[-1]
            break



    member_record.update(current_position)
    member_records.append(member_record)


In [5]:
members_df=pd.DataFrame.from_records(member_records)

In [6]:
class ContentData:
    def __init__(self, article_id):
        self.article_id = article_id
        self.json_path = self._find_jsons_path(article_id)

    def _find_jsons_path(self, article_id):
        content_dir = "../group-data/website_data/content/"
        for file_name in os.listdir(content_dir):
            file_path = os.path.join(content_dir, file_name)
            if os.path.isfile(file_path):
                with open(file_path, "r") as f:
                    content_data = json.load(f)
                if content_data.get("article_id") == article_id:
                    return file_path

        raise ValueError(f"Article ID '{article_id}' not found.")

    def load_json(self):
        with open(self.json_path, "r") as f:
            data = json.load(f)
        return data


In [7]:
#Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates
environment = Environment(loader=FileSystemLoader('templates/'),extensions=['jinja2.ext.loopcontrols'])
environment.globals['page_link'] = page_link
environment.globals['ContentData'] = ContentData

In [8]:
content_id_data = {"article_id": [], "category": [], "date": [], "tags": []}
content_directory = '../group-data/website_data/content'

for json_file in os.listdir(content_directory):
    if json_file.endswith('.json'):
        json_path = os.path.join(content_directory, json_file)
        with open(json_path, 'r') as file:
            info = json.load(file)
            if info.get('display'):
                content_id_data['article_id'].append(info.get('article_id'))
                content_id_data['category'].append(info.get('category'))
                content_id_data['date'].append(info.get('article_date'))
                content_id_data['tags'].append(info.get('tags'))

content_df = pd.DataFrame(content_id_data)
content_df['date'] = pd.to_datetime(content_df['date'], format='%m-%d-%Y')
content_df = content_df.groupby('category').apply(lambda x: x.sort_values('date', ascending=False)).reset_index(drop=True)

In [9]:
research_content_unsorted = content_df[content_df['tags'].apply(lambda x: any('research' in tag for tag in x))]
research_content = research_content_unsorted.groupby('category').apply(lambda x: x.sort_values('date', ascending=False)).reset_index(drop=True)

<ipython-input-9-317efb93bc43>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  research_content = research_content_unsorted.groupby('category').apply(lambda x: x.sort_values('date', ascending=False)).reset_index(drop=True)


In [10]:
news_content_unsorted = content_df[content_df['tags'].apply(lambda x: any('news' in tag for tag in x))]
news_content = news_content_unsorted.sort_values(by="date", ascending=False)

In [11]:
latest_content_df = pd.DataFrame()

for category in content_df.category.unique():
    latest_data = pd.Series(content_df[content_df.category == category].iloc[0])
    latest_content_df = latest_content_df._append(latest_data, ignore_index=True)

latest_content_df['date'] = pd.to_datetime(latest_content_df['date'], format='%m-%d-%Y')
latest_content_df = latest_content_df.sort_values(by='date', ascending=False)

In [12]:
json_files = ['general', 'homepage', 'research', 'support', 'contact']
data = {}

for json_file in json_files:
    try:
        with open(f"../group-data/website_data/{json_file}.json") as json_var:
            data[json_file] = json.load(json_var)
    except (FileNotFoundError, json.JSONDecodeError):
        pass

##### Homepage

In [13]:
homepage_template = environment.get_template('homepage.html.j2')

In [14]:
homepage_content = \
    homepage_template.render(general=data['general'],
                             homepage=data['homepage'],
                             recent_content=latest_content_df.to_dict(orient='records'))

In [15]:
with open('../kerzendorf-group.github.io/index.html', mode='w', encoding='utf-8') as Homepage:
    Homepage.write(homepage_content)

In [16]:
members_df = pd.DataFrame.from_records(member_records)
print(members_df.info())





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   first_name        23 non-null     object
 1   last_name         23 non-null     object
 2   nick_name         23 non-null     object
 3   image_path        23 non-null     object
 4   cover_image_path  23 non-null     object
 5   introduction      23 non-null     object
 6   id                23 non-null     object
 7   institution       22 non-null     object
 8   city              22 non-null     object
 9   state             21 non-null     object
 10  country           22 non-null     object
 11  role              9 non-null      object
 12  start_date        20 non-null     object
 13  end_date          15 non-null     object
 14  degree            13 non-null     object
 15  subject           12 non-null     object
 16  start-date        1 non-null      object
 17  entry2            

##### People Page

In [17]:
members_df = pd.DataFrame.from_records(member_records)
people_template = environment.get_template("people.html.j2")
people_content = people_template.render(general=data["general"], members=members_df)


In [18]:
people_content = people_template.render(general=data["general"], members=members_df)


In [19]:
with open("../kerzendorf-group.github.io/People.html", mode="w", encoding="utf-8") as people:
    people.write(people_content)

##### Individual People Page

In [20]:
ind_person_template = environment.get_template("individual_person.html.j2")

In [21]:
for person in people_df['id']:
            filename = f"../kerzendorf-group.github.io/members/{ person }/{ person }.html"
            ind_person_content = ind_person_template.render(general=data["general"], 
                                                            member_id=person, 
                                                            content=content_df.to_dict(orient='records'))
            with open(filename, mode="w", encoding="utf-8") as page:
                page.write(ind_person_content)

NameError: name 'people_df' is not defined

##### Research Page

In [ ]:
research_template = environment.get_template("research.html.j2")

In [ ]:
main_page_research_content = research_template.render(general=data["general"],
                                            content=research_content)

In [ ]:
with open("../kerzendorf-group.github.io/Research.html", mode="w", encoding="utf-8") as research:
        research.write(main_page_research_content)

In [ ]:
sub_research_template = environment.get_template("sub_research_frontpage.html.j2")

In [ ]:
for category in content_df.loc[content_df.category != "News", "category"].unique():
        sub_research_content = sub_research_template.render(general=data["general"], 
                                                            research_general=data["research"], 
                                                            content = research_content,
                                                            category = category
                                                            )
        folder_path = f"../kerzendorf-group.github.io/sub_research/{page_link(category.lower())}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"../kerzendorf-group.github.io/sub_research/{page_link(category.lower())}.html", mode="w", encoding="utf-8") as sub_research:
            sub_research.write(sub_research_content)

##### Individual Research Page

In [ ]:
template_no_twitter = environment.get_template("research_page_no_twitter.html.j2")

In [ ]:
for ind_research_keys, ind_research_values in research_content.iterrows():
    if "news" not in ind_research_values.category.lower():
        ind_research_content = template_no_twitter.render(general=data["general"], 
                                                          member_ids = people_df['id'],
                                                          nonmem_ids = non_mem_df['id'],
                                                          content = ind_research_values
                                                          
                                                            )
        folder_path = f"../kerzendorf-group.github.io/sub_research/{page_link(ind_research_values.category.lower())}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{ folder_path }/{page_link(ind_research_values.article_id.lower())}.html", mode="w", encoding="utf-8") as ind_research_page:
            ind_research_page.write(ind_research_content)

##### News Page

In [ ]:
news_content

In [ ]:
news_template = environment.get_template("news.html.j2")

In [ ]:
news_page_content = news_template.render(general=data["general"],
                                         content=news_content,
                                         member_ids=people_df['id'],
                                         nonmem_ids = non_mem_df['id'],
                                         category="News")

In [ ]:
with open("../kerzendorf-group.github.io/News.html", mode="w", encoding="utf-8") as news:
        news.write(news_page_content)

##### Individual News Pages

In [ ]:
news_template_no_twitter = environment.get_template("news_page_no_twitter.html.j2")
#news_template_twitter = environment.get_template("news_page_twitter.html.j2")

In [ ]:
for ind_news_keys, ind_news_values in news_content.iterrows():
        ind_news_content = news_template_no_twitter.render(general=data["general"], 
                                                          member_ids = people_df['id'],
                                                          nonmem_ids = non_mem_df['id'],
                                                          content = ind_news_values
                                                            )
        folder_path = f"../kerzendorf-group.github.io/news/"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{ folder_path }/{page_link(ind_news_values.article_id.lower())}.html", mode="w", encoding="utf-8") as ind_news_page:
            ind_news_page.write(ind_news_content)

##### Support Page

In [ ]:
support_template = environment.get_template('support.html.j2')

In [ ]:
support_content = support_template.render(general=data["general"], support=data["support"])

In [ ]:
with open('../kerzendorf-group.github.io/Support.html', mode='w', encoding='utf-8') as support:
    support.write(support_content)

##### Contact

In [ ]:
contact_template = environment.get_template('contact.html.j2')

In [ ]:
contact_content = contact_template.render(general=data["general"], contact=data["contact"])

In [ ]:
with open('../kerzendorf-group.github.io/Contact.html', mode='w', encoding='utf-8') as contact:
    contact.write(contact_content)